# Developed path for each MMSI/Vessel on selection
The list shows all the unique MMSI number and their respective information such as Destination and ETA if available. Upon selection of any MMSI number it generates the path with starting point as green icon and ending point as red and the path itself is shown as blue dots. 

In [17]:
import pandas as pd
import folium
import ipywidgets as widgets
from IPython.display import display, clear_output
from folium.plugins import AntPath  # For animated direction arrows

# Load compiled AIS data
compiled_df = pd.read_csv("compiled_ais_data.csv")

# Ensure required columns exist
required_columns = {'MMSI', 'Latitude', 'Longitude', '# Timestamp', 'Destination', 'ETA'}
if not required_columns.issubset(compiled_df.columns):
    raise ValueError(f"Missing required columns: {required_columns - set(compiled_df.columns)}")

# Convert Timestamp column to datetime
compiled_df['# Timestamp'] = pd.to_datetime(compiled_df['# Timestamp'])

# Extract unique vessel details
vessel_info_list = []
for mmsi, group in compiled_df.groupby('MMSI'):
    destination = group['Destination'].dropna().iloc[0] if not group['Destination'].dropna().empty else "Unknown"
    eta = group['ETA'].dropna().iloc[0] if not group['ETA'].dropna().empty else "Unknown"
    total_positions = len(group)
    vessel_info_list.append((mmsi, destination, eta, total_positions))

# Convert list to DataFrame
vessel_info_df = pd.DataFrame(vessel_info_list, columns=['MMSI', 'Destination', 'ETA', 'Total Positions'])

# Create a formatted list for dropdown options
dropdown_options = {
    f"{row['MMSI']} | {row['Destination']} | {row['ETA']} | {row['Total Positions']} positions": row['MMSI']
    for _, row in vessel_info_df.iterrows()
}

# Create a scrollable dropdown
mmsi_dropdown = widgets.Dropdown(
    options=dropdown_options,
    description="Select Vessel:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')  # Expand dropdown width for better readability
)

# Function to display vessel journey with first & last points highlighted
def display_vessel_journey(selected_mmsi):
    clear_output(wait=True)  # Clear previous output before updating
    display(mmsi_dropdown)  # Redisplay dropdown

    # Filter and sort data for the selected vessel
    vessel_df = compiled_df[compiled_df['MMSI'] == selected_mmsi].sort_values(by='# Timestamp')

    # Extract first and last positions
    first_position = vessel_df.iloc[0]
    last_position = vessel_df.iloc[-1]

    # Extract vessel details
    vessel_info = vessel_info_df[vessel_info_df['MMSI'] == selected_mmsi].iloc[0]
    print(f"Vessel Details for MMSI {selected_mmsi}:")
    print(f"Destination: {vessel_info['Destination']}")
    print(f"ETA: {vessel_info['ETA']}")
    print(f"Total Recorded Positions: {vessel_info['Total Positions']}\n")

    # Create a Folium map centered at the first position
    vessel_map = folium.Map(location=[first_position['Latitude'], first_position['Longitude']], zoom_start=7)

    # Plot the vessel's journey with directional arrows using AntPath
    path_coordinates = vessel_df[['Latitude', 'Longitude']].values.tolist()
    AntPath(locations=path_coordinates, delay=400, weight=2, color="blue").add_to(vessel_map)

    # Plot the journey as blue dots
    for _, row in vessel_df.iterrows():
        folium.CircleMarker(location=[row['Latitude'], row['Longitude']], radius=3, color='blue', fill=True).add_to(vessel_map)

    # Mark first position in green (Start)
    folium.Marker(
        location=[first_position['Latitude'], first_position['Longitude']],
        popup="Start Position",
        icon=folium.Icon(color="green", icon="play")
    ).add_to(vessel_map)

    # Mark last position in red (End)
    folium.Marker(
        location=[last_position['Latitude'], last_position['Longitude']],
        popup="End Position",
        icon=folium.Icon(color="red", icon="stop")
    ).add_to(vessel_map)

    # Display the map
    display(vessel_map)

# Attach the function to the dropdown
widgets.interactive(display_vessel_journey, selected_mmsi=mmsi_dropdown)


interactive(children=(Dropdown(description='Select Vessel:', layout=Layout(width='600px'), options={'186805396…